In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC 
from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from keras.layers import Dense, Input, Conv2D, MaxPool2D, UpSampling2D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from keras.models import Model
from imgaug import augmenters
import pandas as pd
import keras


from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

!pip install split-folders
!rm -rf output

!git clone https://github.com/pruvi007/ML_Datasets.git
    
import split_folders
split_folders.ratio('ML_Datasets/UCMerced_LandUse/Images', output="output", seed=1337, ratio=(.8, .2)) 

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'output'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_file_name = 'alexnet-transfer.pt'
#len(class_names)

model = torchvision.models.alexnet(pretrained=True)

from torchsummary import summary
model.features
#summary(model.features.cuda(), (3,256,256))

for param in model.parameters():
    param.requires_grad = False

model = model.features.to(device)

batch_size=32
train_size, validation_size = 1680, 420
def extract_features(phase, sample_count):
    features = np.zeros(shape=(sample_count, 256, 7, 7))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
   
    # Pass data through convolutional base
    i = 0
    for inputs, label in dataloaders[phase]:
        inputs = inputs.to(device)
        features_batch = model(inputs)
        features[i * batch_size: (i + 1) * batch_size] = features_batch.cpu().numpy()
        labels[i * batch_size: (i + 1) * batch_size] = label.numpy()
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
  
train_features, train_labels = extract_features('train', train_size) 
validation_features, validation_labels = extract_features('val', validation_size)

def scale(X, eps = 0.001):
	# scale the data points s.t the columns of the feature space
	# (i.e the predictors) are within the range [0, 1]
	return (X - np.min(X, axis = 0)) / (np.max(X, axis = 0) + eps)

X_train, y_train = train_features.reshape(1680,7*7*256), train_labels
X_train = scale(X_train)
X_test = scale(validation_features.reshape(420,7*7*256))
y_test = validation_labels
X_test.shape

import keras
y_train = keras.utils.to_categorical(y_train,21)
y_test = keras.utils.to_categorical(y_test,21)

## input layer
input_layer = Input(shape=(12544,))

## encoding architecture
encode_layer1 = Dense(200, activation='relu')(input_layer)
# encode_layer2 = Dense(5000, activation='relu')(encode_layer1)
# encode_layer3 = Dense(500, activation='relu')(encode_layer2)

## latent view
latent_view   = Dense(12544, activation='sigmoid')(encode_layer1)

## decoding architecture
# decode_layer1 = Dense(500, activation='relu')(latent_view)
# decode_layer2 = Dense(5000, activation='relu')(decode_layer1)
# decode_layer3 = Dense(15000, activation='relu')(decode_layer2)

## output layer
output_layer  = Dense(21, activation='softmax')(latent_view)

autoencoder = Model(input_layer,latent_view)

autoencoder.compile(optimizer='adam', loss='mse')
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
# autoencoder.fit(principalDf, principalDf, epochs=20, batch_size=32, validation_data=(principalDf1,principalDf1 ),verbose=1)
autoencoder.fit(scale(X_train), scale(X_train), epochs=10, batch_size=32, validation_data=(scale(X_test), scale(X_test) ),verbose=1)

model = Model(input_layer, output_layer)
model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])
model_log = model.fit(scale(X_train), y_train, epochs=20, batch_size=32, validation_data=(scale(X_test), y_test ),verbose=1)



fatal: destination path 'ML_Datasets' already exists and is not an empty directory.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1680 samples, validate on 420 samples
Epoch 1/10
1680/1680 [==============================] - 7s 4ms/step - loss: 0.0398 - val_loss: 0.0254
Epoch 2/10
1680/1680 [==============================] - 6s 4ms/step - loss: 0.0160 - val_loss: 0.0251
Epoch 3/10
1680/1680 [==============================] - 6s 4ms/step - loss: 0.0158 - val_loss: 0.0249
Epoch 4/10
1680/1680 [==============================] - 6s 4ms/step - loss: 0.0156 - val_loss: 0.0247
Epoch 5/10
1680/1680 [==============================] - 6s 4ms/step - loss: 0.0155 - val_loss: 0.0245
Epoch 6/10
1680/1680 [==============================] - 6s 4ms/step - loss: 0.0153 - val_loss: 0.0244
Epoch 7/10
1680/1680 [==============================] - 6s 4ms/step - loss: 0.0152 - val_loss: 0.0242
Epoch 8/10
1680/1680 [=============